# Satellite Acquisition

**BEFORE RUNNING THIS CODE:
test.bin needs to be generated using gps-sdr-sim, using the command below. https://github.com/osqzss/gps-sdr-sim The brdc2000.15n file is from: ftp://cddis.gsfc.nasa.gov/gnss/data/daily/

This notebook demonstrates the satellite acquisition proccess, in which we determine which satellites are overhead. This code implements the parallel code phase search algorithm, which is the most complex of those available, but is also the fastest since it returns a result in just one iteration. It is possible that we will need to use a different algorithm when the code is running on an embedded platform.

As noted in the L-12 GPS data README, the sampling frequency is 4.092 MHz. and the IF is 0. Also note that the L1 signal hits the antenna at the 1.57542 GHz frequency (10.23 MHz x 154). This is dealt with by the hardware frontend, so we only have to concern ourselves with the 1.023 MHz BPSK modulated C/A code.

## Preamble

In [14]:
import numpy as np
import matplotlib.pyplot as plt
import struct
import math
import heapq

##Grabs the GoldCode module from the root of the repo
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
from GoldCode import GoldCode
from GPSData import IQData

## Import data File

test.bin needs to be generated using gps-sdr-sim, using the command below. 
https://github.com/osqzss/gps-sdr-sim
The brdc2000.15n file is from: 
ftp://cddis.gsfc.nasa.gov/gnss/data/daily/

In [15]:
# ## Import data File
# Generated from gps-sdr-sim using the command: 
#  ./gps-sdr-sim -e brdc2000.15n -l 43.798582,-120.648512,10.0 -t 2015/07/19,11:17:11 -s 4092000 -d 1 -b 8 -o test.bin

# In[2]:
print("Opening a file.")
fHandle = open('../../gps-sdr-sim/test.bin','rb')
I1 = fHandle.read(1)
Q1 = fHandle.read(1)
print(I1)
print(Q1)
print(int.from_bytes(I1, byteorder='big', signed=True))
print(int.from_bytes(Q1, byteorder='big', signed=True))

i = 1
ms = 10
IData = []
QData = []
while I1 != "":
    IData.append(int.from_bytes(I1, byteorder='big', signed=True))
    QData.append(int.from_bytes(Q1, byteorder='big', signed=True))
    I1 = fHandle.read(1)
    Q1 = fHandle.read(1)
    i = i+1
    if i > ms*4092:
        break
fHandle.close()
print("File is now closed.")

fs = 4097000
t = np.linspace(0,1,len(IData),endpoint=True) 
print(t)
CData = np.zeros(len(IData), dtype=np.complex)
CData = np.array(IData) + 1j*np.array(QData)

Opening a file.


FileNotFoundError: [Errno 2] No such file or directory: '../../gps-sdr-sim/test.bin'

# View Complex Data Spectrum

We need to take the fft of the IQ datastream.

In [16]:
# Since we are using circular-convolution, nfft must be equal to the number of samples
nfft = len(CData)
deltaFreq = fs/nfft
print("Calculated deltaFreq: %f [Hz]" %(deltaFreq))

# Generate frequency lists for plotting fft
# Can use f for regular fft and fshift for shifted version
fs_kHz = fs/1000 #Make easier to read
f = np.linspace (0,fs_kHz,nfft, endpoint=False)
fshift = np.linspace(-fs_kHz/2,fs_kHz/2,nfft,endpoint=False)

SampleLength = ms*10**(-3)
t = np.linspace(0,SampleLength,nfft,endpoint=True) 
# Reference data size section of book to pick the len param. Should be based sample rate
ffC = np.fft.fft(CData, nfft)

NameError: name 'CData' is not defined

# Plot output

In [17]:
plt.plot(f, np.abs(ffC))

plt.title("FFT of Complex Data")
plt.xlabel("Frequency (kHz)")
plt.show()

NameError: name 'f' is not defined

# Get a Gold Code

- Gold Code must be generated as though it were sampled at the sampling rate
- 1023 chips that must take 1ms to pass through (due to chipping rate)
- We are sampling @ 4.092 MHz
- So we need the GC to repeat each chip 4 times before going to the next chip.
- The numpy.repeat() function does this exactly

## SVs in this data:

The satellites are: 
Start time = 2015/07/19,11:17:11 (1854:40631)

sat|  ?    |  ?   |   ?         | ?
---|-------|------|-------------|-----
01 | 209.8 | 14.0 | 24397817.3  | 3.7
03 | 219.7 | 77.1 | 20309263.6  | 1.5
06 | 317.5 | 14.0 | 24309631.9  | 3.7
07 | 227.3 |  1.9 | 25733447.1  | 4.9
09 | 284.7 | 25.1 | 23238199.5  | 2.9
16 | 136.5 | 31.5 | 22442996.6  | 2.6
23 | 300.5 | 59.6 | 21068804.2  | 1.7
26 |  99.5 | 40.9 | 21945145.6  | 2.2
31 |  49.8 | 31.8 | 22795607.4  | 2.6
32 | 147.0 | 43.2 | 22071954.8  | 2.1


In [18]:
#code = np.repeat(CodeGen.getCode(1023), 4)

#Choose which satellite's C/A code is generated
Satellite = 15

# Create list of C/A code Taps, for simpler sat selection",
#sat = [(2,6),(3,7),(4,8),(5,9),(1,9),(2,6),(1,8),(2,9),(3,10),(2,3),(3,4),(5,6),(6,7),(7,8),(8,9),(9,10),(1,4),(2,5),(3,6),(4,7),(5,8),(6,9),(1,3),(4,6),(5,7),(6,8),(7,9),(8,10),(1,6),(2,7),(3,8),(4,9),(5,10),(4,10),(1,7),(2,8),(4,10)]
sat = [(1,5),(2,6),(3,7),(4,8),(0,8),(1,5),(0,7),(1,8),(2,9),(1,2),(2,3),(4,5),(5,6),(6,7),(7,8),(8,9),(0,3),(1,4),(2,5),(3,6),(4,7),(5,8),(0,2),(3,5),(4,6),(5,7),(6,8),(7,9),(0,5),(1,6),(2,7),(3,8),(4,9),(3,9),(0,6),(1,7),(3,9)]

# Create Code Generator object for chosen Satellite
CodeGen = GoldCode(sat[Satellite - 1]) # Index starts at zero

# Generate CA Code
CACode = CodeGen.getCode(1023)

# Reformat GC to have -1 in place of 0 (Now done in GoldCode)
#for n,i in enumerate(CACode):
#    if i==0:
#        CACode[n]=-1
        
# Repeat each chip 4 times (See markdown in above cell), to match our ADC sample frequency",
CACodeSampled = np.repeat(CACode,4)
print("Satellite chosen: %d, with tap: %s" %(Satellite,str(sat[Satellite - 1])))

# Repeat entire array for each ms of data sampled
CACodeSampled = np.tile(CACodeSampled,int(SampleLength*1000))
print(len(CACodeSampled))

Satellite chosen: 15, with tap: (7, 8)


NameError: name 'SampleLength' is not defined

In [19]:
Codefft = np.fft.fft(CACodeSampled, nfft)

plt.plot(f, np.abs(Codefft))
plt.title("FFT of Gold Code")
plt.xlabel("Frequency (kHz)")
plt.show()

NameError: name 'nfft' is not defined

## Function to find second largest value

In [20]:
# This will return second largest value
# It will also ignore any value that is close to the second largest value
def GetSecondLargest(DataList):
    # Make sure is a numpy array
    DataArray = np.array(DataList)
    
    # Find largest value
    Largest = np.amax(DataArray)
    LargestIndex = np.argmax(DataArray)
#    print("Largest value: %f, at position: %d"%(Largest,LargestIndex))
    
    # Reduce value by a percent to prevent near-identical values from being selected
    ScaleAmount = 0.95
    ScaledLargest = ScaleAmount*Largest
    SecondLargest = 0
    SecondLargestIndex = 0
    
    for ind,val in enumerate(DataArray):
        if val < ScaledLargest:
            if val > SecondLargest:
                #Ignore adjacent bins to Largest
                if (np.abs(LargestIndex-ind) > 10):
                    SecondLargest = val
                    SecondLargestIndex = ind
    
#    print("Second largest value: %f, at position: %d"%(SecondLargest,SecondLargestIndex))
    return SecondLargest

# Complex Conjugate of Gold Code fft

Take the complex conjugate of the Gold Code [real(fft) - imag(fft)j], and then multiply it by the IQ fft. If this is done correctly there should be a spike about $10^7$ times as high as the noise. 



In [21]:
GCConj = np.conjugate(Codefft)

result = np.fft.ifft(GCConj * ffC,nfft)

resultSQ = np.real(result*np.conjugate(result))

t = np.linspace(0,SampleLength,nfft,endpoint=True) 
print("Length of t: %d" %(len(t)))
print("Length of IFFT: %d" %(len(resultSQ)))

plt.plot(t,resultSQ)

# Zoom in to one ms
xmin,xmax = plt.xlim()
plt.xlim(xmin,xmin + 0.001)

plt.title("Cross-correlation without adjusting for doppler shift.")
plt.xlabel("Time (seconds)")
plt.show()

# Search for largest value in 1 ms worth of data
SecondLargestValue = GetSecondLargest(resultSQ[0:int(fs*0.001)])

# Pseudo SNR
ResultsSNR = 10*np.log10(np.amax(resultSQ)/SecondLargestValue)
print("SNR: %f"%(ResultsSNR))

print("Max Value: %f, at freqshift: %d, with index %d"%(np.amax(resultSQ),0,np.argmax(resultSQ)))

NameError: name 'Codefft' is not defined

# Notes

- From book appendix: we should look for the top 2 peaks, and use the ratio to determine whether there is a signal or just noise
- We should get the C code running and be sure to compare results. Acquisition works in the C code, but tracking is untested

## Shift frequency using multiplication by a complex exponential

In [22]:
freqShift = 1400 #[Hz] -20


# Initialize complex array
CDataShifted = CData*np.exp(1j*2*np.pi*freqShift*t)
fftCDataShifted = np.fft.fft(CDataShifted,nfft)

result = np.fft.ifft(GCConj * fftCDataShifted,nfft)
resultSQ = np.real(result*np.conjugate(result))
resultLog = 10*np.log10(resultSQ)

plt.plot(t,resultSQ)
plt.title("Cross-correlation with doppler adjustment of %d [Hz]"%(freqShift))
plt.xlabel("Time (seconds)")

# Zoom in to one ms
xmin,xmax = plt.xlim()
plt.xlim(xmin,xmin + 0.001)
plt.show()

# Search for largest value in 1 ms worth of data
SecondLargestValue = GetSecondLargest(resultSQ[0:int(fs*0.001)])

# Pseudo SNR
ResultsSNR = 10*np.log10(np.amax(resultSQ)/SecondLargestValue)
print("SNR: %f"%(ResultsSNR))

print("Max Value: %f, at freqshift: %d, with index: %d"%(np.amax(resultSQ),freqShift,np.argmax(resultSQ)))

NameError: name 'CData' is not defined

##### Shift frequency by rotating FFT array

In [23]:
freqRoll = -1 #[Bin number (See BinWidth for frequency resolution)]

# Initialize complex array
fftCDataShifted = np.roll(np.fft.fft(CData,nfft),freqRoll)

result = np.fft.ifft(GCConj * fftCDataShifted,nfft)
resultSQ = np.real(result*np.conjugate(result))
resultLog = 20*np.log10(np.abs(result))

BinWidth = fs/nfft
plt.plot(t,resultSQ)
plt.title("Cross-correlation with doppler adjustment of %f [Hz]"%(freqRoll*BinWidth))
plt.xlabel("Time (seconds)")

# Zoom in to one ms
xmin,xmax = plt.xlim()
plt.xlim(xmin,xmin + 0.001)
plt.show()

# Search for largest value in 1 ms worth of data
SecondLargestValue = GetSecondLargest(resultSQ[0:int(fs*0.001)])

# Pseudo SNR
ResultsSNR = 10*np.log10(np.amax(resultSQ)/SecondLargestValue)
print("SNR: %f"%(ResultsSNR))

print("Max Value: %f, at freqshift: %d, with index: %d"%(np.amax(resultSQ),freqShift,np.argmax(resultSQ)))

NameError: name 'CData' is not defined

## Shift through frequency and satellite range
The signal was generated with gps-sdr-gen with command: 
  ./gps-sdr-sim -e brdc2000.15n -l 43.798582,-120.648512,10.0 -t 2015/07/19,11:17:11 -s 4092000 -d 1 -b 8 -o test.bin

The satellites are: 
Start time = 2015/07/19,11:17:11 (1854:40631)

sat|  ?    |  ?   |   ?         | ?
---|-------|------|-------------|-----
01 | 209.8 | 14.0 | 24397817.3  | 3.7
03 | 219.7 | 77.1 | 20309263.6  | 1.5
06 | 317.5 | 14.0 | 24309631.9  | 3.7
07 | 227.3 |  1.9 | 25733447.1  | 4.9
09 | 284.7 | 25.1 | 23238199.5  | 2.9
16 | 136.5 | 31.5 | 22442996.6  | 2.6
23 | 300.5 | 59.6 | 21068804.2  | 1.7
26 |  99.5 | 40.9 | 21945145.6  | 2.2
31 |  49.8 | 31.8 | 22795607.4  | 2.6
32 | 147.0 | 43.2 | 22071954.8  | 2.1


In [24]:
# Choose what frequencies and satellites to increment over
StartingFrequencyShift = -10*10**3
EndingFrequencyShift = 10*10**3
FrequencyShiftIncrement = 100
FrequencyList = range(StartingFrequencyShift,EndingFrequencyShift + FrequencyShiftIncrement,FrequencyShiftIncrement)

StartingSatellite = 1
EndingSatellite = 32
SatelliteList = range(StartingSatellite, EndingSatellite + 1)

# Create list of C/A code Taps, for simpler sat selection",
#sat = [(2,6),(3,7),(4,8),(5,9),(1,9),(2,6),(1,8),(2,9),(3,10),(2,3),(3,4),(5,6),(6,7),(7,8),(8,9),(9,10),(1,4),(2,5),(3,6),(4,7),(5,8),(6,9),(1,3),(4,6),(5,7),(6,8),(7,9),(8,10),(1,6),(2,7),(3,8),(4,9),(5,10),(4,10),(1,7),(2,8),(4,10)]
sat = [(1,5),(2,6),(3,7),(4,8),(0,8),(1,5),(0,7),(1,8),(2,9),(1,2),(2,3),(4,5),(5,6),(6,7),(7,8),(8,9),(0,3),(1,4),(2,5),(3,6),(4,7),(5,8),(0,2),(3,5),(4,6),(5,7),(6,8),(7,9),(0,5),(1,6),(2,7),(3,8),(4,9),(3,9),(0,6),(1,7),(3,9)]

# Create array to store max values, freq ranges, per satellite
SatMax = np.zeros((len(SatelliteList),len(FrequencyList),4))

satInd = 0 
# Loop through selected satellites
for curSat in SatelliteList: 
    # Create Code Generator object for chosen Satellite
    CodeGen = GoldCode(sat[curSat - 1]) # Index starts at zero
    
    # Generate CA Code
    CACode = CodeGen.getCode(1023)
    
    # Reformat GC to have -1 in place of 0
    for n,i in enumerate(CACode):
        if i==0:
            CACode[n]=-1
            
    # Repeat each chip 4 times (See markdown in above cell), to match our ADC sample frequency",
    CACodeSampled = np.repeat(CACode,4)
    #print("Satellite chosen: %d, with tap: %s" %(Satellite,str(sat[Satellite - 1])))
    
    # Repeat entire array for each ms of data sampled
    CACodeSampled = np.tile(CACodeSampled,ms)
    Codefft = np.fft.fft(CACodeSampled,nfft)
    GCConj = np.conjugate(Codefft)
    
    freqInd = 0
    # Loop through all frequencies
    for curFreq in FrequencyList:
        # Initialize complex array
        CDataShifted = np.zeros(len(CData), dtype=np.complex)
        
        # Shift frequency using complex exponential 
        CDataShifted = CData*np.exp(1j*2*np.pi*curFreq*t)

        fftCDataShifted = np.fft.fft(CDataShifted,nfft)
        
        result = np.fft.ifft(GCConj * fftCDataShifted,nfft)
        
        resultSQ = np.abs(result) ** 2
        resultSQ = np.real(resultSQ)
        maxAbsSquared = np.max(resultSQ)
        
        rmsPowerdB = 10*np.log10(np.mean(resultSQ))
        
        resultLog= 10*np.log10(resultSQ)
        
        maxdB = np.max(resultLog)
        
                
        # Store results in array
        SatMax[satInd,freqInd,0] = maxAbsSquared
        SatMax[satInd,freqInd,1] = maxdB
        SatMax[satInd,freqInd,2] = np.real(np.sum(resultLog))
        SatMax[satInd,freqInd,3] = maxdB - rmsPowerdB
        
        #print("Sat: %d, Doppler Shift: %8.2f, MaxVal: %8.2f, MaxVal (dB): %8.4f, Peak-Second Ratio (dB): %8.4f" %(curSat,curFreq, maxAbsSquared, maxdB, maxdB - rmsPowerdB))

        freqInd = freqInd + 1
        
    plt.figure()
    
    plt.plot(FrequencyList, SatMax[satInd, :, 3])
    #plt.ylim((0,20))
    plt.xlabel('Doppler Shift (Hz)')
    plt.ylabel('Peak-to-RMS ratio (dB)')
    plt.title("Sat %d"%curSat)
    plt.show()
    
    satInd = satInd+1

#maxAbsSquared
#secondMax

NameError: name 'ms' is not defined

For this data sample (Launch 12, 4 seconds after launch) we should see the following satellites:

Sat | S/N (dB-Hz) | doppler shift (Hz) | code phase (chips)
:----- | -------:| -------:| --------:
 1 | 13.287713  | -1209.276720 | 394.250000
 4 | 13.700614 | 600.950356 | 547.750000
 11 | 13.381992 | 612.980296 | 514.750000
 13 | 13.554054 | -12.064141 | 655.250000
 15 | 15.276798 | -1399.910091 | 489.750000
 
 So if we generate the Gold codes for the correct satellites and correct for the Doppler shift, we should see a peak in the time domain correlator.


# Generate Gold Code for selected satellite

In [25]:
sats = {1: -1209.3, 4: 600.95, 11: 612.98, 13: -12.06, 15: -1399.9}

plt.style.use('ggplot')

for Satellite, Doppler in sats.items():

    #code = np.repeat(CodeGen.getCode(1023), 4)

    # Create Code Generator object for chosen Satellite
    CodeGen = GoldCode(sat[Satellite - 1]) # Index starts at zero

    # Generate CA Code
    CACode = CodeGen.getCode(1023)

    # Reformat GC to have -1 in place of 0 (Now done in GoldCode)
    #for n,i in enumerate(CACode):
    #    if i==0:
    #        CACode[n]=-1

    # Repeat each chip 4 times (See markdown in above cell), to match our ADC sample frequency",
    CACodeSampled = np.repeat(CACode,4)

    # Repeat entire array for each ms of data sampled
    CACodeSampled = np.tile(CACodeSampled,int(SampleLength*1000))
 
    Codefft = np.fft.fft(CACodeSampled)
    GCConj = np.conjugate(Codefft)
    

    result = np.fft.ifft(GCConj * ffC,nfft)

    resultSQ = np.real(result*np.conjugate(result))

    t = np.linspace(0,SampleLength,nfft,endpoint=True) 
#    plt.figure(figsize=(10,6))
    
    plt.plot(t,resultSQ)

    # Zoom in to one ms
    xmin,xmax = plt.xlim()
    plt.xlim(xmin,xmin + 0.001)


    plt.title("SV%d Cross-correlation without\nadjusting for Doppler shift."%Satellite)
    plt.xlabel("Time (seconds)")
    plt.show()

    # Search for largest value in 1 ms worth of data
    SecondLargestValue = GetSecondLargest(resultSQ[0:int(fs*0.001)])

    # Pseudo SNR
    ResultsSNR = 10*np.log10(np.amax(resultSQ)/SecondLargestValue)
    
    freqShift = -Doppler #[Hz] -20


    # Initialize complex array
    CDataShifted = CData*np.exp(1j*2*np.pi*freqShift*t)
    fftCDataShifted = np.fft.fft(CDataShifted,nfft)

    result = np.fft.ifft(GCConj * fftCDataShifted,nfft)
    resultSQ = np.real(result*np.conjugate(result))
    resultLog = 10*np.log10(resultSQ)

 #   plt.figure(figsize=(10,6))
    plt.plot(t,resultSQ)
    plt.title("SV%d Cross-correlation with\nDoppler adjustment of %d [Hz]"%(Satellite, freqShift))
    plt.xlabel("Time (seconds)")

    # Zoom in to one ms
    xmin,xmax = plt.xlim()
    plt.xlim(xmin,xmin + 0.001)
    plt.show()

    # Search for largest value in 1 ms worth of data
    SecondLargestValue = GetSecondLargest(resultSQ[0:int(fs*0.001)])

    # Pseudo SNR
    ResultsSNR = 10*np.log10(np.amax(resultSQ)/SecondLargestValue)


NameError: name 'SampleLength' is not defined

In [26]:
maxAbsSquared

NameError: name 'maxAbsSquared' is not defined

In [ ]:
secondMax

In [ ]:
resultSQ

In [ ]:
plt.plot(resultSQ)
plt.show()